In [72]:
import fitz  # PyMuPDF
import re
import spacy
import pymupdf
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pdfplumber
import pandas as pd
from tqdm import tqdm

In [10]:
dataframe=pd.read_csv("DataSet.csv")
dataframe.head()
dataframe.paragraphs.isna().sum()

0

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import ngrams
from collections import Counter
from nltk.corpus import stopwords
# nltk.download('all')

# Example sentence

def extract_all_bigrams_and_trigrams(dataframe):
# Tokenize the sentence
        list_bigrams = []
        list_trigrams = []
        stop_words = set(stopwords.words('english'))
        for i in range(len(dataframe)):
            tokens = word_tokenize(dataframe.paragraphs[i].lower())
            # Remove stop words from tokens
            filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
            # Generate bigrams (n=2)
            bigrams = list(ngrams(filtered_tokens, 2))
            
            # print("Bigrams:", bigrams[:10])

            # Generate trigrams (n=3)
            trigrams = list(ngrams(filtered_tokens, 3))
            
            # print("Trigrams:", trigrams[:10])

            # Count the frequency of bigrams
            bigram_freq = Counter(bigrams)
            # print("Bigram Frequencies:", bigram_freq)
            trigrams_freq = Counter(trigrams)
            list_bigrams.append(bigram_freq)
            list_trigrams.append(trigrams_freq)
        return list_bigrams,list_trigrams
            # print("trigrams Frequencies:", trigrams_freq)

In [12]:
import spacy
from collections import Counter

# Load spaCy English model for POS tagging
nlp = spacy.load("en_core_web_lg")

# Step 1: Function to filter bigrams containing only nouns
def filter_ngrams_nouns(bigram_counter):
    filtered_bigrams = Counter()  # Initialize an empty Counter to store filtered bigrams
    
    for bigram, freq in bigram_counter.items():
        doc = nlp(" ".join(bigram))  # POS tag the bigram
        
        # Check if both tokens in the bigram are nouns
        if all(token.pos_ == "NOUN" for token in doc):
            filtered_bigrams[bigram] = freq  # Add only noun-based bigrams to the new Counter
    
    return filtered_bigrams


# # Step 2: Apply the filter to keep only noun-based bigrams
# filtered_bigrams = filter_ngrams_nouns(dataframe.trigram_dict[0])

# # Display the result
# print(filtered_bigrams)

In [13]:
list_bigrams,list_trigrams=extract_all_bigrams_and_trigrams(dataframe)
list_filtered_bigrams = []
list_filtered_trigrams = []
for counter in list_bigrams:
    filtered_bigrams = filter_ngrams_nouns(counter)
    list_filtered_bigrams.append(filtered_bigrams)
for counter in list_trigrams:
    filtered_trigrams = filter_ngrams_nouns(counter)
    list_filtered_trigrams.append(filtered_trigrams)

print(list_filtered_bigrams)
print(list_filtered_trigrams)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
[Counter({('risk', 'management'): 19, ('project', 'risk'): 14, ('project', 'management'): 3, ('management', 'practice'): 2, ('management', 'processes'): 2, ('plan', 'risk'): 2, ('risk', 'analysis'): 2, ('purpose', 'practice'): 1, ('management', 'practitioners'): 1, ('practitioners', 'stakeholders'): 1, ('stakeholders', 'deﬁ'): 1, ('aspects', 'project'): 1, ('practice', 'projects'): 1, ('projects', 'time'): 

In [14]:
dataframe['bigram_dict'] = list_filtered_bigrams
dataframe['trigram_dict'] = list_filtered_trigrams

In [15]:
def combine_all_bigrams(df, bigram_col):
    combined_counter = Counter()
    
    for dic_bigram in df[bigram_col]:
        combined_counter.update(dic_bigram)
    
    return combined_counter

combine_all_bigrams(dataframe, "bigram_dict")

Counter({('risk', 'management'): 378,
         ('project', 'risk'): 295,
         ('project', 'management'): 168,
         ('risk', 'analysis'): 78,
         ('management', 'process'): 52,
         ('plan', 'risk'): 50,
         ('management', 'plan'): 50,
         ('project', 'objectives'): 48,
         ('risk', 'responses'): 45,
         ('risk', 'identiﬁ'): 40,
         ('risk', 'response'): 38,
         ('management', 'processes'): 36,
         ('project', 'manager'): 25,
         ('project', 'team'): 22,
         ('project', 'stakeholders'): 21,
         ('technique', 'strengths'): 20,
         ('probability', 'impact'): 19,
         ('analysis', 'process'): 19,
         ('breakdown', 'structure'): 18,
         ('time', 'cost'): 17,
         ('management', 'project'): 17,
         ('control', 'risks'): 16,
         ('ed', 'risks'): 16,
         ('risk', 'owner'): 16,
         ('management', 'activities'): 15,
         ('risk', 'breakdown'): 15,
         ('trigger', 'conditions'): 

In [16]:
combine_all_bigrams(dataframe, "trigram_dict")

Counter({('project', 'risk', 'management'): 248,
         ('risk', 'management', 'process'): 49,
         ('risk', 'management', 'plan'): 32,
         ('risk', 'management', 'processes'): 24,
         ('technique', 'strengths', 'weaknesses'): 20,
         ('project', 'management', 'plan'): 18,
         ('risk', 'management', 'activities'): 14,
         ('risk', 'breakdown', 'structure'): 14,
         ('risk', 'analysis', 'process'): 13,
         ('risk', 'management', 'technique'): 13,
         ('management', 'technique', 'strengths'): 13,
         ('risk', 'management', 'planning'): 12,
         ('project', 'management', 'processes'): 11,
         ('risk', 'responses', 'process'): 10,
         ('introduction', 'project', 'risk'): 7,
         ('effect', 'project', 'objectives'): 7,
         ('risk', 'action', 'owner'): 7,
         ('risk', 'management', 'project'): 6,
         ('effectiveness', 'project', 'risk'): 6,
         ('decision', 'tree', 'analysis'): 6,
         ('level', 'con

In [9]:
dataframe.head()

,Unnamed: 0,sections,ids,paragraphs,bigram_dict,trigram_dict
0,0,1.1 Purpose of the Practice Standard for Proje...,4,The purpose of the Practice Standard for Pro...,"Counter({('risk', 'management'): 19, ('project...","Counter({('project', 'risk', 'management'): 14..."
1,1,1.2 Project Risk Management Deﬁ nition,66,"The deﬁ nition of Project Risk Management, as...","Counter({('project', 'risk'): 5, ('risk', 'man...","Counter({('project', 'risk', 'management'): 4,..."
2,2,1.3 Role of Project Risk Management in Project...,79,Project Risk Management is not an optional ac...,"Counter({('risk', 'management'): 12, ('project...","Counter({('project', 'risk', 'management'): 8,..."
3,3,1.4 Good Risk Management Practice,124,Project Risk Management is a valuable compone...,"Counter({('risk', 'management'): 16, ('project...","Counter({('project', 'risk', 'management'): 15..."
4,4,1.5 Critical Success Factors for Project Risk ...,157,Figure 1-2. Critical Success Factors for Proj...,"Counter({('risk', 'management'): 15, ('project...","Counter({('project', 'risk', 'management'): 12..."


In [71]:
dataframe.to_csv('dataframe.csv')

In [21]:
def combine_bigrams_trigrams(df):
    combined_counter = Counter()
    
    # Combine all bigrams and trigrams from each row
    for idx, row in df.iterrows():
        bigrams = row['bigram_dict']  # Access the Counter object directly
        trigrams = row['trigram_dict']
        
        # Update the combined Counter with bigrams and trigrams
        combined_counter.update(bigrams)
        combined_counter.update(trigrams)
    
    return combined_counter

In [24]:
combined_ngrams = combine_bigrams_trigrams(dataframe)
combined_ngrams

Counter({('risk', 'management'): 378,
         ('project', 'risk'): 295,
         ('project', 'risk', 'management'): 248,
         ('project', 'management'): 168,
         ('risk', 'analysis'): 78,
         ('management', 'process'): 52,
         ('plan', 'risk'): 50,
         ('management', 'plan'): 50,
         ('risk', 'management', 'process'): 49,
         ('project', 'objectives'): 48,
         ('risk', 'responses'): 45,
         ('risk', 'identiﬁ'): 40,
         ('risk', 'response'): 38,
         ('management', 'processes'): 36,
         ('risk', 'management', 'plan'): 32,
         ('project', 'manager'): 25,
         ('risk', 'management', 'processes'): 24,
         ('project', 'team'): 22,
         ('project', 'stakeholders'): 21,
         ('technique', 'strengths'): 20,
         ('technique', 'strengths', 'weaknesses'): 20,
         ('probability', 'impact'): 19,
         ('analysis', 'process'): 19,
         ('project', 'management', 'plan'): 18,
         ('breakdown', 'struc

In [25]:
def get_top_ngrams(ngram_counter, top_n=20):
    return ngram_counter.most_common(top_n)

In [29]:
top_ngrams = get_top_ngrams(combined_ngrams, top_n=50)
top_ngrams

[(('risk', 'management'), 378),
 (('project', 'risk'), 295),
 (('project', 'risk', 'management'), 248),
 (('project', 'management'), 168),
 (('risk', 'analysis'), 78),
 (('management', 'process'), 52),
 (('plan', 'risk'), 50),
 (('management', 'plan'), 50),
 (('risk', 'management', 'process'), 49),
 (('project', 'objectives'), 48),
 (('risk', 'responses'), 45),
 (('risk', 'identiﬁ'), 40),
 (('risk', 'response'), 38),
 (('management', 'processes'), 36),
 (('risk', 'management', 'plan'), 32),
 (('project', 'manager'), 25),
 (('risk', 'management', 'processes'), 24),
 (('project', 'team'), 22),
 (('project', 'stakeholders'), 21),
 (('technique', 'strengths'), 20),
 (('technique', 'strengths', 'weaknesses'), 20),
 (('probability', 'impact'), 19),
 (('analysis', 'process'), 19),
 (('project', 'management', 'plan'), 18),
 (('breakdown', 'structure'), 18),
 (('time', 'cost'), 17),
 (('management', 'project'), 17),
 (('control', 'risks'), 16),
 (('ed', 'risks'), 16),
 (('risk', 'owner'), 16),


In [30]:
df =dataframe.copy()

In [46]:
import re

def remove_special_characters_and_brackets(text):
    # Remove content inside brackets, including the brackets themselves
    text_no_brackets = re.sub(r"\[.*?\]|\(.*?\)|\{.*?\}", "", text)
    
    # Regular expression to match special characters except punctuation
    pattern = r"[^a-zA-Z0-9\s.,!?;:]"
    
    # Replace special characters with an empty string
    cleaned_text = re.sub(pattern, "", text_no_brackets)
    
    return cleaned_text


In [47]:
remove_special_characters_and_brackets(df['paragraphs'][0])

' The purpose of the  Practice Standard for Project Risk Management  is to  provide a standard for project  management practitioners and other stakeholders that de nes the aspects of Project Risk Management that  are recognized as good practice on most projects most of the time and  provide a standard that is globally  applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for  training or educational purposes.   The  Practice Standard for Project Risk Management  covers risk management as it is applied to single  projects only. Like the  PMBOK    Guide   Fourth Edition, this practice standard does not cover risk in programs  or portfolios of projects.   Chapter 11 of the  PMBOK    Guide   Fourth Edition, is the basis for the  Practice Standard for Project Risk  Management . This practice standard is consistent with that chapter, emphasizing the concepts and principles  relating to Project Risk Management. It is aligned with other PMI

In [48]:
df['processed_paragraphs'] = df['paragraphs'].apply(remove_special_characters_and_brackets)

In [70]:
def get_entities(sent):
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""

    #############################################################

    for tok in nlp(sent):
        # Skip punctuation marks
        if tok.dep_ != "punct":
            # Handle compound words (e.g., "risk management")
            if tok.dep_ == "compound":
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text

            # Handle modifiers (e.g., "important project")
            if tok.dep_.endswith("mod"):
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text

            # Detect subject entities
            if "subj" in tok.dep_:
                ent1 = (modifier + " " + prefix + " " + tok.text).strip()
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""

            # Detect object entities
            if "obj" in tok.dep_:
                ent2 = (modifier + " " + prefix + " " + tok.text).strip()

            # Update previous token info
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text

    #############################################################
    
    # Return cleaned entity strings
    return [ent1.strip(), ent2.strip()]


In [71]:
get_entities(" Project Risk Management is not an optional activity: it is essential to successful project management.")

['optional  it', 'successful project management']

In [51]:
def segment_paragraph_into_sentences(paragraph):
    doc = nlp(paragraph)
    sentences = [sent.text for sent in doc.sents]
    return sentences

# Apply the segmentation to each paragraph in the DataFrame
df['sentences'] = df['processed_paragraphs'].apply(segment_paragraph_into_sentences)


In [52]:
df['sentences'][0] 

[' The purpose of the  Practice Standard for Project Risk Management  is to  provide a standard for project  management practitioners and other stakeholders that de nes the aspects of Project Risk Management that  are recognized as good practice on most projects most of the time and  provide a standard that is globally  applicable and consistently applied.',
 'This practice standard has a descriptive purpose rather than one used for  training or educational purposes.   ',
 'The  Practice Standard for Project Risk Management  covers risk management as it is applied to single  projects only.',
 'Like the  PMBOK    Guide   Fourth Edition, this practice standard does not cover risk in programs  or portfolios of projects.   ',
 'Chapter 11 of the  PMBOK    Guide   Fourth Edition, is the basis for the  Practice Standard for Project Risk  Management .',
 'This practice standard is consistent with that chapter, emphasizing the concepts and principles  relating to Project Risk Management.',
 'I

In [77]:
df.head()

,sections,ids,paragraphs,bigram_dict,trigram_dict,relations,sentences,processed_paragraphs
0,1.1 Purpose of the Practice Standard for Proje...,4,The purpose of the Practice Standard for Pro...,"{('purpose', 'practice'): 1, ('project', 'risk...","{('purpose', 'practice', 'standard'): 1, ('pro...",[],[ The purpose of the Practice Standard for Pr...,The purpose of the Practice Standard for Pro...
1,1.2 Project Risk Management Deﬁ nition,66,"The deﬁ nition of Project Risk Management, as...","{('nition', 'project'): 1, ('project', 'risk')...","{('nition', 'project', 'risk'): 1, ('project',...",[],"[ The de nition of Project Risk Management, as...","The de nition of Project Risk Management, as ..."
2,1.3 Role of Project Risk Management in Project...,79,Project Risk Management is not an optional ac...,"{('project', 'risk'): 11, ('risk', 'management...","{('project', 'risk', 'management'): 8, ('proje...",[],[ Project Risk Management is not an optional a...,Project Risk Management is not an optional ac...
3,1.4 Good Risk Management Practice,124,Project Risk Management is a valuable compone...,"{('project', 'risk'): 15, ('risk', 'management...","{('project', 'risk', 'management'): 15, ('comp...",[],[ Project Risk Management is a valuable compon...,Project Risk Management is a valuable compone...
4,1.5 Critical Success Factors for Project Risk ...,157,Figure 1-2. Critical Success Factors for Proj...,"{('success', 'factors'): 2, ('project', 'risk'...","{('success', 'factors', 'project'): 2, ('proje...",[],"[ Figure 12., Critical Success Factors for Pro...",Figure 12. Critical Success Factors for Proje...


In [79]:
df.drop(['ids','relations'] ,axis=1, inplace=True)

In [74]:
Entity_pairs =[]
for sents in tqdm(df["sentences"][0]):
    Entity_pairs.append(get_entities(sents))
len(Entity_pairs)




















100%|██████████| 32/32 [00:00<00:00, 152.13it/s]


32

In [80]:
df.to_csv('df.csv')

In [75]:

entity_pairs = []
for i in range (len(df["sentences"])):
  for i in tqdm(df["sentences"][i]):
    entity_pairs.append(get_entities(i))
entity_pairs























































100%|██████████| 32/32 [00:00<00:00, 146.36it/s]



















100%|██████████| 6/6 [00:00<00:00, 127.12it/s][A




































100%|██████████| 28/28 [00:00<00:00, 176.73it/s]



















100%|██████████| 15/15 [00:00<00:00, 155.41it/s]


















100%|██████████| 16/16 [00:00<00:00, 183.21it/s]



















100%|██████████| 4/4 [00:00<00:00, 128.74it/s][A


















100%|██████████| 3/3 [00:00<00:00, 104.38it/s][A



















100%|██████████| 21/21 [00:00<00:00, 191.08it/s]


















100%|██████████| 10/10 [00:00<00:00, 150.77it/s]



















100%|██████████| 12/12 [00:00<00:00, 154.60it/s]


















100%|██████████| 8/8 [00:00<00:00, 151.26it/s][A



















100%|██████████| 8/8 [00:00<00:00, 158.23it/s][A


















100%|██████████| 8/8 [00:00<00:00, 165.01it/s][A



















100%|██████████| 17/17 [00:00<00:00, 190.94it/s]



[['most  that', 'most  standard'],
 ['practice standard', 'rather  training'],
 ['risk it', 'single  projects'],
 ['PMBOK practice standard', 'projects'],
 ['Chapter', '11 Project Risk Management'],
 ['practice standard', 'Project Risk Management'],
 ['It', 'other PMI practice standards'],
 ['Figure', '11 Fourth Edition'],
 ['', ''],
 ['', 'Hierarchy PMI Project'],
 ['', 'Practice  Project'],
 ['practice standard', 'PMBOK Fourth Edition'],
 ['', ''],
 ['', 'six Fourth Edition'],
 ['six  processes', 'Quantitative Risk Analysis'],
 ['six  that', 'critical success process'],
 ['', ''],
 ['which', 'practice standard'],
 ['that', 'effective Project Risk Management'],
 ['principles', 'several  reasons'],
 ['Principles', 'now  future'],
 ['constantly  principles', 'more  stability'],
 ['Different  projects', 'different Project Risk Management'],
 ['that', 'large  projects'],
 ['more  practice', 'organizational  culture'],
 ['different risk that', 'practice standard'],
 ['', ''],
 ['joint  ven

In [54]:
from spacy.matcher import Matcher 
from spacy.tokens import Span 
def get_relation(sent):
    doc = nlp(sent)

    # Matcher class object 
    matcher = Matcher(nlp.vocab)

    # Define the pattern
    pattern = [{'DEP': 'ROOT'}, 
               {'DEP': 'prep', 'OP': "?"},
               {'DEP': 'agent', 'OP': "?"},  
               {'POS': 'ADJ', 'OP': "?"}]

    # Add the pattern to the matcher
    matcher.add("matching_1", [pattern])

    matches = matcher(doc)
    
    if matches:
        k = len(matches) - 1  # Get the last match
        span = doc[matches[k][1]:matches[k][2]]  # Extract the matched span
        return span.text
    else:
        return None

In [76]:
list_of_relations = []
for sents in df['sentences'][0]:
    list_of_relations.append(get_relation(sents))

print(len(list_of_relations)) 

list_of_relations

32


['is',
 'has',
 'covers',
 'cover',
 'is',
 'is consistent',
 'aligned with other',
 'compares',
 'Figure',
 'Institute',
 'INTRODUCTION',
 'organized in',
 '2',
 'Principles',
 'are',
 'described in',
 '3',
 'glossary of',
 'emphasizes',
 'can',
 'expected',
 'have more',
 'require different',
 'is',
 'be',
 'are many',
 '3',
 'found on',
 'used as',
 'establish',
 'Institute',
 'Practice']

In [58]:
for i in range (len(df["sentences"])):
    for sents in df['sentences'][i]:
        list_of_relations.append(tqdm(get_relation(sents)))


  0%|          | 0/2 [00:00<?, ?it/s]

In [63]:
len(list_of_relations)

1200

In [62]:
pd.Series(list_of_relations).value_counts()[:50]

is                                                                       3
3                                                                        2
Institute                                                                2
(p, r, o, c, e, s, s)                                                    1
(i, n, c, l, u, d, e, d)                                                 1
(i, n, c, l, u, d, e)                                                    1
(i, n, c, l, u, d, e)                                                    1
(r, e, q, u, i, r, e, d)                                                 1
(m, i, r, r, o, r)                                                       1
(b, e,  , a, p, p, l, i, c, a, b, l, e)                                  1
(P, r, a, c, t, i, c, e)                                                 1
(b, e, n, e)                                                             1
(I, n, s, t, i, t, u, t, e)                                              1
(n, e, e, d)             